# pySCA 7.0 - PDZ Domain Example

This notebook demonstrates the complete SCA workflow using the PDZ domain alignment (PF00595).

**Dataset:** PDZ_PF00595_aln.fasta  
**Database:** Outputs/PDZ_PF00595_aln.db.gz

**Workflow Steps:**
1. Load existing processed database
2. Run SCA core calculations with independent component identification
3. Visualize results using Plotly
4. Explore independent components

**How to Use:**
- Execute cells one by one using `Shift+Enter` or the "Run" button
- Cells must be executed in order (top to bottom)
- You can re-run individual cells after the initial setup
- Interactive Plotly plots will appear inline below each visualization cell

**Author:** pySCA Team  
**Date:** 2026


## Setup and Imports


In [1]:
# Check for required dependencies
try:
    import plotly
    print("✓ plotly is installed")
except ImportError:
    print("✗ plotly is NOT installed")
    print("\nTo install, run in terminal:")
    print("  conda activate pysca3")
    print('  pip install -e ".[notebooks]"')
    print("\nOr install just plotly:")
    print("  pip install plotly>=5.0.0")
    raise

import os
import numpy as np
from pathlib import Path

# Import pySCA utilities
from pysca import notebook_utils as nb
from pysca import scaTools as sca

# Import Plotly for interactive visualizations
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Configure Plotly for Cursor/VS Code compatibility
# In Cursor/VS Code, we need to use a renderer that displays HTML inline
try:
    import plotly.io as pio
    # Try to use 'vscode' renderer if available, otherwise use 'plotly_mimetype'
    # This allows plots to display inline in VS Code/Cursor notebooks
    if 'vscode' in pio.renderers:
        pio.renderers.default = "vscode"
        print("✓ Plotly configured for VS Code/Cursor display")
    elif 'plotly_mimetype' in pio.renderers:
        pio.renderers.default = "plotly_mimetype"
        print("✓ Plotly configured for notebook display")
    else:
        # Fallback: use default which should work in most cases
        pio.renderers.default = "notebook"
        print("✓ Plotly configured (using default renderer)")
except Exception as e:
    print(f"Note: Plotly renderer configuration: {e}")
    print("Plots may open in browser instead of inline")

# Set up paths - navigate to project root if running from notebooks/
current_dir = Path(os.getcwd())
if current_dir.name == "notebooks":
    project_root = current_dir.parent
else:
    project_root = current_dir

output_dir = project_root / "Outputs"
db_path = output_dir / "PDZ_PF00595_aln.db.gz"

print(f"✓ All imports successful")
print(f"✓ Working directory: {os.getcwd()}")
print(f"✓ Project root: {project_root}")
print(f"✓ Database path: {db_path}")
print(f"✓ Database exists: {db_path.exists()}")
print("\nSetup complete!")


✓ plotly is installed


/opt/miniconda3/envs/pysca3/lib/python3.11/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


✓ Plotly configured for VS Code/Cursor display
✓ All imports successful
✓ Working directory: /Users/rama/cAI/pySCA/notebooks
✓ Project root: /Users/rama/cAI/pySCA
✓ Database path: /Users/rama/cAI/pySCA/Outputs/PDZ_PF00595_aln.db.gz
✓ Database exists: True

Setup complete!


## Step 1: Load and Inspect Processed Database

First, let's load the database that was created by `sca-process-msa` and see what's in it.


In [2]:
# Load the database
if db_path.exists():
    db = nb.load_database(db_path)
    
    # Print summary
    nb.print_summary(db)
    
    # Access specific data
    seq_data = db["sequence"]
    print(f"\n{'='*60}")
    print("Sequence Data Details:")
    print(f"{'='*60}")
    # Use correct database keys: Nseq (not M), effseqs (not M_eff), Npos (not L)
    print(f"Number of sequences: {seq_data.get('Nseq', 'N/A')}")
    print(f"Number of effective sequences: {seq_data.get('effseqs', 'N/A')}")
    if isinstance(seq_data.get('effseqs'), (int, float)):
        print(f"  (M_eff = {seq_data['effseqs']:.2f})")
    print(f"Number of positions: {seq_data.get('Npos', 'N/A')}")
    print(f"Reference sequence index: {seq_data.get('i_ref', 'N/A')}")
    if 'hd' in seq_data and seq_data.get('i_ref') is not None:
        i_ref = seq_data['i_ref']
        if isinstance(i_ref, (int, np.integer)) and i_ref < len(seq_data['hd']):
            print(f"Reference sequence header: {seq_data['hd'][i_ref]}")
else:
    print(f"ERROR: Database file not found at {db_path}")
    print("Please run sca-process-msa first to create the database.")


Database Summary

Sequence Data:
  M           : 13854
  M_eff       : 13854.0
  L           : 74
  i_ref       : 14406

SCA Data:
  Csca_shape  : (74, 74)
  has_eigenvalues: True
  has_simMat  : True
  Ntrials     : 10

Sector/IC Data:
  kpos        : 8
  kpos_auto   : 12
  n_ics       : 4
  has_Vica    : True

Sequence Data Details:
Number of sequences: 13854
Number of effective sequences: 13854.0
  (M_eff = 13854.00)
Number of positions: 74
Reference sequence index: 14406


## Step 2: Run SCA Core Calculations

Now let's run the SCA core calculations. If the database already has SCA results, we can skip this step. Otherwise, we'll compute:
- Positional weights (Di, Dia)
- SCA matrix (Csca)
- Eigendecomposition (V, L)
- Randomization trials
- Independent component identification


In [3]:
# Check if SCA calculations have already been done
if "sca" in db and "Csca" in db["sca"]:
    print("SCA core calculations already present in database.")
    print("Skipping sca-core step. To recompute, delete the database and rerun sca-core.")
else:
    print("Running SCA core calculations...")
    # Run SCA core calculations with independent component identification
    output_db_path = nb.run_sca_core(
        database=db_path,
        norm="frob",  # Frobenius norm for matrix reduction
        Ntrials=10,  # Number of randomization trials
        lbda=0.01,  # Regularization parameter
        do_seqcorr=False,  # Set to True to compute sequence correlations (memory-intensive)
        do_sector_id=True,  # Perform independent component identification
        kpos=None,  # None = auto-select number of eigenmodes
        kica=None,  # None = use kpos for ICA
        sector_cutoff=0.95,  # T-distribution cutoff percentile
        kmax_cap=10,  # Maximum kpos when auto-selecting
        matlab=True,  # Also write MATLAB workspace
        verbose=True
    )
    
    print(f"\nSCA core calculations complete!")
    print(f"Updated database: {output_db_path}")
    
    # Reload database with new SCA data
    db = nb.load_database(output_db_path)


SCA core calculations already present in database.
Skipping sca-core step. To recompute, delete the database and rerun sca-core.


## Step 3: Visualize Results

Now let's visualize the SCA results using interactive Plotly plots.


### 3.1 Eigenvalue Spectrum

The eigenvalue spectrum shows the strength of positional correlations. Compare the SCA eigenvalues (blue) with randomized alignments (orange) to identify significant modes.


In [ ]:
# Plot eigenvalues with randomized comparison (Plotly - interactive)
if "sca" in db and "L" in db["sca"]:
    fig = nb.plot_eigenvalues_plotly(
        db, 
        show_randomized=True,
        height=500,
        width=900
    )
    fig.show()
    
    # Print some statistics
    L = db["sca"]["L"]
    print(f"\nTop 10 eigenvalues:")
    for i in range(min(10, len(L))):
        print(f"  Mode {i+1}: {L[i]:.4f}")
else:
    print("Eigenvalues not found. Run sca-core first.")



Top 10 eigenvalues:
  Mode 1: 14.1325
  Mode 2: 4.7973
  Mode 3: 4.4048
  Mode 4: 3.6416
  Mode 5: 3.4237
  Mode 6: 3.3189
  Mode 7: 3.1345
  Mode 8: 3.0095
  Mode 9: 2.6870
  Mode 10: 2.6606


### 3.2 Independent Component Heatmap

View all independent components as a heatmap to see which positions contribute to each IC.


In [6]:
# Plot heatmap of all independent components (Plotly - interactive)
if "sector" in db and "Vica" in db["sector"]:
    fig = nb.plot_ic_heatmap_plotly(
        db, 
        ic_index=None,  # Show all ICs
        height=600, 
        width=1000,
        colorscale="RdBu"
    )
    fig.show()
    
    # Print IC information
    sector_data = db["sector"]
    print(f"\nNumber of independent components: {len(sector_data.get('ic_list', []))}")
    print(f"Number of eigenmodes used (kpos): {sector_data.get('kpos', 'N/A')}")
    print(f"Auto-estimated kpos: {sector_data.get('kpos_auto', 'N/A')}")
else:
    print("Independent components not found. Run sca-core with --do-sector-id first.")



Number of independent components: 4
Number of eigenmodes used (kpos): 8
Auto-estimated kpos: 12


### 3.3 Individual IC Analysis

Examine individual independent components to see which positions are most significant.


In [7]:
# Plot individual ICs with significant positions highlighted
if "sector" in db and "Vica" in db["sector"]:
    sector_data = db["sector"]
    n_ics = len(sector_data.get("ic_list", []))
    
    print(f"Plotting first {min(3, n_ics)} independent components...\n")
    
    for ic_idx in range(min(3, n_ics)):
        print(f"IC {ic_idx + 1}:")
        fig = nb.plot_ic_positions_plotly(
            db, 
            ic_index=ic_idx, 
            top_n=20,
            height=400,
            width=900
        )
        fig.show()
        
        # Print significant positions
        if "sector_ats" in sector_data and ic_idx < len(sector_data["sector_ats"]):
            ic_ats = sector_data["sector_ats"][ic_idx]
            print(f"  Significant positions ({len(ic_ats)} total): {ic_ats[:15]}...")
        print()
else:
    print("Independent components not found.")


Plotting first 3 independent components...

IC 1:


  Significant positions (10 total): ['362', '328', '351', '322', '341', '376', '386', '350', '338', '345']...

IC 2:


  Significant positions (6 total): ['372', '330', '325', '329', '380', '347']...

IC 3:


  Significant positions (4 total): ['375', '336', '379', '323']...



### 3.4 IC Correlation Matrix

Examine correlations between independent components to understand their relationships.


In [ ]:
# Plot Spearman correlation matrix between ICs (Plotly - interactive)
if "sector" in db and "ic_corr_spearman" in db["sector"]:
    fig = nb.plot_ic_correlation_plotly(
        db,
        height=600,
        width=700,
        colorscale="RdBu"
    )
    fig.show()
    
    # Print correlation statistics
    ic_corr = db["sector"]["ic_corr_spearman"]
    n_ics = len(ic_corr)
    print(f"\nIC Correlation Statistics:")
    print(f"  Number of ICs: {n_ics}")
    print(f"  Mean absolute correlation: {np.abs(ic_corr[np.triu_indices(n_ics, k=1)]).mean():.3f}")
    print(f"  Max correlation: {ic_corr[np.triu_indices(n_ics, k=1)].max():.3f}")
else:
    print("IC correlation matrix not found.")


## Step 4: Explore IC Data Programmatically

Access and analyze the independent component data directly.


In [ ]:
# Access sector/IC data
if "sector" in db:
    sector_data = db["sector"]
    
    print("="*60)
    print("Independent Component Summary")
    print("="*60)
    print(f"Number of eigenmodes (kpos): {sector_data.get('kpos', 'N/A')}")
    print(f"Auto-estimated kpos: {sector_data.get('kpos_auto', 'N/A')}")
    print(f"Number of independent components: {len(sector_data.get('ic_list', []))}")
    
    # Print significant positions for each IC (in ATS numbering)
    if "sector_ats" in sector_data:
        print(f"\nSignificant positions (ATS numbering) for each IC:")
        for i, ic_ats in enumerate(sector_data["sector_ats"]):
            print(f"\n  IC {i+1}: {len(ic_ats)} significant positions")
            if len(ic_ats) > 0:
                # Show all positions (formatted nicely)
                positions_str = "+".join(ic_ats)
                print(f"    Positions: {positions_str}")
else:
    print("Sector data not found.")


## Step 5: Custom Visualizations

Create custom visualizations using the raw data.


In [ ]:
# Compare multiple ICs side-by-side
if "sector" in db and "Vica" in db["sector"]:
    sector_data = db["sector"]
    Vica = sector_data["Vica"]
    n_ics = min(4, Vica.shape[0])  # Show first 4 ICs
    
    # Create subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[f"IC {i+1}" for i in range(n_ics)],
        vertical_spacing=0.12,
        horizontal_spacing=0.1
    )
    
    for i in range(n_ics):
        row = (i // 2) + 1
        col = (i % 2) + 1
        positions = np.arange(Vica.shape[1])
        
        # Get significant positions for highlighting
        sig_pos = set()
        if "sector_pos" in sector_data and i < len(sector_data["sector_pos"]):
            sig_pos = set(sector_data["sector_pos"][i])
        
        # Plot all positions
        fig.add_trace(
            go.Scatter(
                x=positions,
                y=Vica[i, :],
                mode='lines+markers',
                name=f'IC {i+1}',
                line=dict(width=1, color='lightgray'),
                marker=dict(size=2, color='lightgray', opacity=0.5),
                showlegend=False,
                hovertemplate='Position: %{x}<br>Value: %{y:.4f}<extra></extra>'
            ),
            row=row, col=col
        )
        
        # Highlight significant positions
        if sig_pos:
            sig_indices = [p for p in positions if p in sig_pos]
            sig_values = [Vica[i, p] for p in sig_indices]
            fig.add_trace(
                go.Scatter(
                    x=sig_indices,
                    y=sig_values,
                    mode='markers',
                    name=f'IC {i+1} significant',
                    marker=dict(size=6, color='red', line=dict(width=1, color='darkred')),
                    showlegend=False,
                    hovertemplate='Position: %{x}<br>Value: %{y:.4f}<extra></extra>'
                ),
                row=row, col=col
            )
    
    fig.update_layout(
        title_text="Independent Components Comparison",
        height=600,
        width=1000,
        showlegend=False,
        template='plotly_white'
    )
    fig.update_xaxes(title_text="Position (ATS)")
    fig.update_yaxes(title_text="IC Value")
    fig.show()
else:
    print("IC data not available.")


## Summary

This notebook demonstrated:
1. Loading a processed MSA database
2. Running SCA core calculations (if needed)
3. Visualizing eigenvalues, independent components, and correlations
4. Accessing and analyzing IC data programmatically
5. Creating custom visualizations

**Next Steps:**
- Explore individual ICs in more detail
- Map significant positions to structure (if PDB available)
- Compare ICs across different alignments
- Analyze sequence correlations (enable `--do-seqcorr` in sca-core)
